# Prepare data for Figure "DGVM" - Prepare DGVM for regridding

In [ ]:
# Libraries
import os
import numpy as np
import xarray as xr

In [ ]:
# Directories
dir_data = '../data/'
dir05x = '../paper_deficit/output/05_prep_other/fig_dgvm/'

---

In [ ]:
# Create directories for each dgvm if not exists
for i in ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
          'jules', 'lpjguess', 'lpjwsl', 'orchidee']:
    if not os.path.exists(os.path.join(dir05x, i)):
        os.mkdir(os.path.join(dir05x, i))

In [ ]:
# Prepare dgvms mean 2010-2020
def prep_da_model(mdir, m, s, dtype):
    
    """Prepare dgvm model variable"""
    
    # Get data and adjust time if necessary
    if mdir not in ['cablepop', 'dlem', 'ibis', 'isam', 'isbactrip']:
        ds = xr.open_dataset(
            os.path.join(dir_data, 'trendy_v11', mdir, f'{m}_{s}_{dtype}.nc'))
    if mdir in ['cablepop', 'dlem', 'ibis', 'isam', 'isbactrip']:
        ds = xr.open_dataset(
            os.path.join(dir_data, 'trendy_v11', mdir, f'{m}_{s}_{dtype}.nc'), 
            decode_times = False)
        ds['time'] = ds.time + 1700
    # Take mean from 2010 to 2020 and convert to t/ha    
    return (ds.sel(time = slice('2010', '2020')).mean('time')[dtype] * 10)

# Function to prepare model data and export
def prep_ds_model(mdir, m):
    
    """Prepare dgvm, select relevant variables, isam and jules do not have 
    litter adjust coordinate names if necessary"""
    
    # Create dataset with cveg, csoil, and clitter variables of s2 and s3 simulations
    ds = xr.Dataset()
    ds[mdir + '_s2_cveg'] = prep_da_model(mdir, m, 'S2', 'cVeg')
    ds[mdir + '_s3_cveg'] = prep_da_model(mdir, m, 'S3', 'cVeg')
    ds[mdir + '_s2_csoil'] = prep_da_model(mdir, m, 'S2', 'cSoil')
    ds[mdir + '_s3_csoil'] = prep_da_model(mdir, m, 'S3', 'cSoil')
    if mdir not in ['isam', 'jules']:
        ds[mdir + '_s2_clit'] = prep_da_model(mdir, m, 'S2', 'cLitter') 
        ds[mdir + '_s3_clit'] = prep_da_model(mdir, m, 'S3', 'cLitter')
    # Rename coordinates
    if mdir in ['cablepop', 'classic', 'ibis', 'jsbach', 'lpjguess', 
                'orchidee']:
        ds = ds.rename(latitude='lat', longitude='lon')
    # Export
    ds.to_netcdf(os.path.join(dir05x, mdir, f'ds_{mdir}.nc'), mode='w')

# Prepare model data
prep_ds_model('cablepop', 'CABLE-POP')
prep_ds_model('classic', 'CLASSIC')
prep_ds_model('clm', 'CLM5.0')
prep_ds_model('dlem', 'DLEM')
prep_ds_model('ibis', 'IBIS')
prep_ds_model('isam', 'ISAM')
prep_ds_model('jsbach', 'JSBACH')
prep_ds_model('jules', 'JULES')
prep_ds_model('lpjguess', 'LPJ-GUESS')
prep_ds_model('lpjwsl', 'LPJ')
prep_ds_model('orchidee', 'ORCHIDEE')

#['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 'jules', 
# 'lpjguess', 'lpjwsl', 'orchidee']

# cardamom: exclude  because of time and year dimensions, which are not interpretable, and missing cSoil
# isbactrip: exclude because of strange names of dimensions and variables
# isam: included but no litter
# jules: included but no litter

In [ ]:
# Create land_sea_masks
for m in ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
          'jules', 'lpjguess', 'lpjwsl', 'orchidee']:

    # Get data
    ds = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}.nc'))

    # Create land sea mask and export
    xr.where(~np.isnan(ds[m + '_s2_cveg']), 1, 0) \
        .astype('float64') \
        .rename('land_sea_mask') \
        .to_netcdf(os.path.join(dir05x, m, f'ds_{m}_land.nc'), mode='w')

In [ ]:
# Prepare dgvms (no mean)
def prep_da_model(mdir, m, s, dtype):
   
    """Prepare dgvm model variable"""
    
    # Get data and adjust time if necessary
    if mdir not in ['cablepop', 'dlem', 'ibis', 'isam', 'isbactrip']:
        ds = xr.open_dataset(
            os.path.join(dir_data, 'trendy_v11', mdir, f'{m}_{s}_{dtype}.nc'))
        ds['time'] = [i.year for i in 
                      ds.time.values.astype('datetime64[D]') \
                      .astype(object)]
    if mdir in ['cablepop', 'dlem', 'ibis', 'isam', 'isbactrip']:
        ds = xr.open_dataset(
            os.path.join(dir_data, f'trendy_v11/{mdir}/{m}_{s}_{dtype}.nc'),
            decode_times = False)
        ds['time'] = (ds.time + 1700).astype('int')
    # Convert to t/ha    
    return (ds[dtype] * 10)


# Prepare model data and export
def prep_ds_model(mdir, m):
    
    """Prepare dgvm, select relevant variables, isam and jules do not have 
    litter adjust coordinate names if necessary"""
    
    # Create dataset with cveg, csoil, and clitter variables of s2 and s3 simulations
    ds = xr.Dataset()
    ds[mdir + '_s2_cveg'] = prep_da_model(mdir, m, 'S2', 'cVeg')
    ds[mdir + '_s3_cveg'] = prep_da_model(mdir, m, 'S3', 'cVeg')
    ds[mdir + '_s2_csoil'] = prep_da_model(mdir, m, 'S2', 'cSoil')
    ds[mdir + '_s3_csoil'] = prep_da_model(mdir, m, 'S3', 'cSoil')
    
    if mdir not in ['isam', 'jules']:
        ds[mdir + '_s2_clit'] = prep_da_model(mdir, m, 'S2', 'cLitter') 
        ds[mdir + '_s3_clit'] = prep_da_model(mdir, m, 'S3', 'cLitter')
    # Rename coordinates
    if mdir in ['cablepop', 'classic', 'ibis', 'jsbach', 'lpjguess', 
                'orchidee']:
        ds = ds.rename(latitude='lat', longitude='lon')
    # Export
    ds.to_netcdf(os.path.join(dir05x, mdir, f'ds_{mdir}_1700_2021.nc'), 
                              mode='w')


# Prepare model data
prep_ds_model('cablepop', 'CABLE-POP')
prep_ds_model('classic', 'CLASSIC')
prep_ds_model('clm', 'CLM5.0')
prep_ds_model('dlem', 'DLEM')
prep_ds_model('ibis', 'IBIS')
prep_ds_model('isam', 'ISAM')
prep_ds_model('jsbach', 'JSBACH')
prep_ds_model('jules', 'JULES')
prep_ds_model('lpjguess', 'LPJ-GUESS')
prep_ds_model('lpjwsl', 'LPJ')
prep_ds_model('orchidee', 'ORCHIDEE')

#['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 'jules', 
# 'lpjguess', 'lpjwsl', 'orchidee']

# cardamom: exclude  because of time and year dimensions, which are not interpretable, and missing cSoil
# isbactrip: exclude because of strange names of dimensions and variables
# isam: included but no litter
# jules: included but no litter

---